# RNN Based molucule generation

Laurent Cetinsoy

In this hands-on we want to generate molecule formulas for denovo-drug discovery.

For that we need to use Generative models. Generative models are models which goes beyond classification or simple regression : they are able to generate data that look like previously seens dataset.

There exists a lot of models :

- Bayesian models like graphical models
- Recurrent models (for sequence generation like texte)
- Variational auto encoders
- Generative adversarial models
- Flow and diffusion models


In the hands-on we will start by  trainning a character based RNN to generate smile molecules


We want to feed smile representations of molecules to an RNN.
The basic idea is we will train it to predict the next smile token of a molecule given the previous one.

For instance for the following molecule "CC(=O)NC1=CC=C(O)C=C1" will may give to the model

X = "CC(=O)N"
y = C

and ask the RNN to learn to predict y given X

Like a standard language model !


## RNN Language model


A language model is a model which predict the next token of a sequence given the previous ones :

$ P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p})  $


This model can be learned with a Recurrent neural network

$ y = P(X_t | X_{t-1}, X_{t-2}, ..., X_{t-p}) = RNN_{\theta} (X_{t-1}, X_{t-2}, ..., X_{t-p})  $


In order to train such model you need a corpus of data.



There are two main ways to do that : Word level model or character level model

For character level models, an interesting resource is : http://karpathy.github.io/2015/05/21/rnn-effectiveness/



Explain briefly what is the difference between word based language model and character based language model

Les modèles basés sur des caractères fonctionnent sur la prédiction de la lettre la plus probable de suivre une séquence. Les modèles basés sur des mots fonctionnent en revanche sur la prédiction du mot le plus probable.

## Loading the data

Dowload the following dataset : https://github.com/joeymach/Leveraging-VAE-to-generate-molecules

In [2]:
! [ -e 250k_smiles.csv ] || wget https://raw.githubusercontent.com/joeymach/Leveraging-VAE-to-generate-molecules/master/250k_smiles.csv

Import pandas and load the first 1000 lines

In [3]:
import pandas as pd
import numpy as np
import random

In [4]:
df = pd.read_csv(filepath_or_buffer="250k_smiles.csv", nrows=1000)

Display the first rows of the dataframe

In [5]:
df.head()

,smiles,logP,qed,SAS
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1\n,5.05060,0.702012,2.084095
1,C[C@@H]1CC(Nc2cncc(-c3nncn3C)c2)C[C@@H](C)C1\n,3.11370,0.928975,3.432004
2,N#Cc1ccc(-c2ccc(O[C@@H](C(=O)N3CCCC3)c3ccccc3)...,4.96778,0.599682,2.470633
3,CCOC(=O)[C@@H]1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c...,4.00022,0.690944,2.822753
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])[C@H](C#...,3.60956,0.789027,4.035182


## Processing the data

We need to do the following things :

- convert smile tokens to numbers
- build  smile token sequences and corresponding labels pairs

Compute the biggest smile molecule size

In [6]:
len(max(df["smiles"], key=lambda s: len(s)))

106

Code a function unic_characters(string) which return the unic characters in a string

In [7]:
def unic_characters(string):
    return np.unique(list(string))

In [8]:
unic_characters("AAAABAAACCCDDDEE")

array(['A', 'B', 'C', 'D', 'E'], dtype='<U1')

Concatenate all smile string of the pandas dataframe and use **unic_characters** to get the unic_characters

In [9]:
unic_chars = unic_characters(df["smiles"].sum())

Code a function **map_char_to_int(unic_chars)** which returns a dictionnary where each char is assigned an int value.
Add a character to specify the end of the molecule (like "\n")


In [10]:
def map_char_to_int(unic_chars):
    dictionnary = {}
    for i, char in enumerate(unic_chars):
        dictionnary[char] = i
    return dictionnary

In [11]:
map_char_to_int(unic_chars)

{'\n': 0,
 '#': 1,
 '(': 2,
 ')': 3,
 '+': 4,
 '-': 5,
 '/': 6,
 '1': 7,
 '2': 8,
 '3': 9,
 '4': 10,
 '5': 11,
 '6': 12,
 '7': 13,
 '=': 14,
 '@': 15,
 'B': 16,
 'C': 17,
 'F': 18,
 'H': 19,
 'I': 20,
 'N': 21,
 'O': 22,
 'S': 23,
 '[': 24,
 '\\': 25,
 ']': 26,
 'c': 27,
 'l': 28,
 'n': 29,
 'o': 30,
 'r': 31,
 's': 32}

Code a function map_int_to_char(unic_chars) which returns the reverse mapping.

If you want you can merge both functions in a class

In [12]:
def map_int_to_char(unic_chars):
    dictionnary = {}
    for i, char in enumerate(unic_chars):
        dictionnary[i] = char
    return dictionnary

In [13]:
class MolEncoder:
    def __init__(self, unic_characters: np.array):
        self.unic_chars = unic_characters
        self.char_to_int = map_char_to_int(unic_chars)
        self.int_to_char = map_int_to_char(unic_chars)
        self.voc_len = len(unic_chars)

    def get_char(self, int_val):
        return self.int_to_char[int_val]

    def get_int(self, char):
        return self.char_to_int[char]

    def encode_mol(self, smiles):
        return np.array([self.char_to_int[char] for char in smiles])

    def get_one_hot(self, int_value: int = None, char: chr = None):
        if int_value is None:
            int_value = self.get_int(char)
        elif char is None:
            char = self.get_char(int_value)
        one_hot = np.zeros(self.voc_len)
        one_hot[int_value] = 1
        return one_hot

    def decode_mol(self, encoded_mol):
        return "".join([self.int_to_char[int] for int in encoded_mol])

In [14]:
mol_encoder = MolEncoder(unic_chars)
print(mol_encoder.get_char(17))
print(mol_encoder.get_int("C"))
print(mol_encoder.get_one_hot(17))
assert np.all(mol_encoder.get_one_hot(17) == mol_encoder.get_one_hot(char="C"))

C
17
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [15]:
mol_smile = df["smiles"][random.randint(0, len(df))]
ecnoded = mol_encoder.encode_mol(mol_smile)
assert mol_encoder.decode_mol(ecnoded) == mol_smile

For each smile molecule add the ending token to it

## Building the dataset

Now we will create the dataset so that it has the good share for our Keras LSTM model

Remember Keras recurrent models expect a 3D array with shapes (n_examples, seq_len, n_features)



What will be `n_features` in our case ?

`n_features` est la dimension de chaque vecteur de caractères. Dans notre cas, il s'agit du nombre de caractères uniques dans le dataset.

Code a function **build_X_and_y(string, i_char, seq_lenght)** which takes a string, a **seq_length** number and a position.


It should create X by by getting all character between i and i + seq_length
and create y by getting the character following the X sequence
it returns X and y

In [16]:
def build_X_and_y(
    string: str,
    i_char: int,
    seq_length: int,
    mol_encoder: MolEncoder = None,
    one_hot: bool = False,
):
    encode_method = (
        mol_encoder.get_int
        if not one_hot
        else lambda x: mol_encoder.get_one_hot(char=x)
    )
    X = [encode_method(char) for char in string[i_char : i_char + seq_length]]
    y = encode_method(string[i_char + seq_length])
    return X, y

Test your function on the following string "OCC(C)(C)c1ccc" with seq_length = 4 and i = [1, 2, 3]

In [196]:
tested_string = "CC(C)(C)c1ccc"
seq_len = 3
for i_char in range(1, 4):
    X, y = build_X_and_y(
        tested_string, i_char=i_char, seq_length=seq_len, mol_encoder=mol_encoder
    )
    print(
        f"l'encodage de {tested_string[i_char:i_char+seq_len]} est {X} et le caractère suivant est {y}"
    )

l'encodage de C(C est [17, 2, 17] et le caractère suivant est 3
l'encodage de (C) est [2, 17, 3] et le caractère suivant est 2
l'encodage de C)( est [17, 3, 2] et le caractère suivant est 17


By using build_X_and_y and map_char_to_int build a list nameed X_train and a list named y_train

In [197]:
X_train, y_train = [], []
for mol in df["smiles"]:
    for i in range(len(mol) - seq_len):
        X, y = build_X_and_y(mol, i_char=i, seq_length=seq_len, mol_encoder=mol_encoder)
        X_train.append(X)
        y_train.append(y)

Create numpy arrays from the lists

In [198]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [199]:
X_train.shape, y_train.shape

((42242, 3), (42242,))

Reshape the X numpy array (n_examples, seq_lenght, 1)

In [200]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

In [201]:
X_train.shape, y_train.shape

((42242, 3, 1), (42242,))

Normalize X by dividing each values by the total number of unic characters

In [202]:
X_train = X_train / mol_encoder.voc_len

In [203]:
assert np.all(X_train < 1)

Import Keras and build (at least) a two layered LSTM network with 128 neurone in each.

You can also add Dropoutlayers

Do you think you should use the return_sequences = True ? If yes, when ?


Add a Dense layer on top with with the appropriate activation function and number of neurones


In [204]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional

In [205]:
X_train.shape

(42242, 3, 1)

In [206]:
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation="sigmoid"))

2023-09-23 19:11:49.321577: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-23 19:11:49.324846: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-23 19:11:49.326819: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Compile the model with the appropriate loss function and the adam optimizer

In [207]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary()

Model: "sequential_43"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_52 (LSTM)              (None, 128)               66560     
                                                                 
 dropout_32 (Dropout)        (None, 128)               0         
                                                                 
 dense_36 (Dense)            (None, 1)                 129       
                                                                 
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


Train the model on 20 epochs and 10 examples (yeah you read correctly) and check that the model overfits !

In [186]:
# Convert X_train to tensor
n_samples = 10
X_train_tensor = tf.convert_to_tensor(X_train[:n_samples])
y_train_tensor = tf.convert_to_tensor(y_train[:n_samples])

In [187]:
model.fit(X_train_tensor, y_train_tensor, epochs=20)

Epoch 1/20


2023-09-23 19:04:31.335041: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-23 19:04:31.338379: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-23 19:04:31.342381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/1 [==============================] - 2s 2s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 2/20
1/1 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 3/20
1/1 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 4/20
1/1 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 5/20
1/1 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 6/20
1/1 [==============================] - 0s 15ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 7/20
1/1 [==============================] - 0s 16ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 8/20
1/1 [==============================] - 0s 14ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 9/20
1/1 [==============================] - 0s 11ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 10/20
1/1 [==============================] - 0s 12ms/step - lo

If it does not overfit try to fix data prep and model architecture so it does

Create a function **make_prediction(seed_start)** which takes a starting string sequence and uses it to generate a molecule


generate a molecule of your overfitted model

Make a model checkpoint so that the model is saved after each epoch
if you train on a plateform and it stops you do not lose your training

Now go to your favorite plateform (colab or something else) and train the dataset on the whole data for 10 epochs and batch size 256

it should take a long time so either follow the class or go take a nap

Generate between 100 and 1000 molecules.

create a list where molecules have between 10 and 50 atoms

With rdkit compute the Quantified Estimated Drug likelyness (QED) of each molecule in this subset

Bonus 1 : Using rdkit, compute the quantitative estimation of drug-likeness (QED) of your generated molecules.

Bonus 2 : try to adapt a transformer model training from hugging face to see if it is better